### Load the Commit Statistics
For this clustering we examined 39 GitHub projects containing a total of 2,629 commits. The projects all included the Fluence Marine library. Code for Fluence is generally written in React and Rust.

In [1]:
project_root_path = '../../..'

import sys
lib_path = project_root_path + "/jupyter/lib/"
sys.path.append(lib_path)

import commit_stat_engineering
import json

dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

### Commit Stats


In [2]:
print(all_stats[0].keys())

dict_keys(['commit', 'totalFiles', 'binFiles', 'textFiles', 'textLines', 'javascriptFiles', 'javascriptLines', 'javascriptFilePct', 'javascriptLinePct', 'rustFiles', 'rustLines', 'rustFilePct', 'rustLinePct', 'markdownFiles', 'markdownLines', 'markdownFilePct', 'markdownLinePct', 'jsonFiles', 'jsonLines', 'jsonFilePct', 'jsonLinePct', 'imgFiles', 'imgFilePct', 'lockFiles', 'lockLines', 'lockFilePct', 'lockLinePct', 'yarnFiles', 'yarnLines', 'yarnFilePct', 'yarnLinePct', 'htmlFiles', 'htmlLines', 'htmlFilePct', 'htmlLinePct', 'clojureFiles', 'clojureLines', 'clojureFilePct', 'clojureLinePct', 'shellFiles', 'shellLines', 'shellFilePct', 'shellLinePct', 'gitignoreFiles', 'gitignoreLines', 'gitignoreFilePct', 'gitignoreLinePct', 'noextbinFiles', 'noextbinFilePct', 'noexttextFiles', 'noexttextLines', 'noexttextFilePct', 'noexttextLinePct'])


### Load Labels

In [3]:
import pandas as pd

data = pd.read_csv(project_root_path + '/sandbox/data/bob/commit_labeling.csv', sep='\t')
data

,Unnamed: 0,wtf,rust_development,interface_development,big_json,rust_packaging,documentation,javascript_development,bulk_commit,minor_commit,...,textFiles,textLines,javascriptLinePct,rustLinePct,markdownLinePct,jsonLinePct,imgFiles,lockLinePct,yarnLinePct,htmlLinePct
0,12074a45bc9b22dfffbf433bb73dde6aad88e2be,1,0,0,0,0,0,0,0,1,...,0,0,0.0,0.00,0.0,0.0,2,0.0,0.00,0.0
1,e190bc9dcda78b5688b25154c1eb6e2e1febfd1c,1,0,0,0,0,0,0,0,1,...,0,0,0.0,0.00,0.0,0.0,0,0.0,0.00,0.0
2,e4807d4413c12c4b9a4600736ebd3642a8e8b2ce,1,0,0,0,0,0,0,0,1,...,0,0,0.0,0.00,0.0,0.0,1,0.0,0.00,0.0
3,5b1f6608a3ae88026b9f2bf317a80287f382e9b7,1,0,0,0,0,0,0,0,1,...,1,1,100.0,0.00,0.0,0.0,0,0.0,0.00,0.0
4,dabe94da0d22f41a555012e6ec4ef689a8c1c6e6,1,0,0,0,0,0,0,0,1,...,1,1,100.0,0.00,0.0,0.0,0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,088905485b5fcabd7c84fd3a7485993a2c5c3d30,1,1,0,0,0,0,0,0,0,...,1,41,0.0,100.00,0.0,0.0,0,0.0,0.00,0.0
156,5ff8d2f8d0f6a0241083616b29687a42e3d0964f,1,1,0,0,0,0,0,0,0,...,5,59,0.0,100.00,0.0,0.0,0,0.0,0.00,0.0
157,91e38c2e82d52be12fb27ece91b4d187ff7540b5,1,1,0,0,0,0,0,0,0,...,3,66,0.0,22.72,0.0,0.0,0,0.0,77.27,0.0
158,6ad2870fafbea1a580320c0835ac122ccaa7736c,1,1,0,0,0,0,0,0,0,...,1,84,0.0,100.00,0.0,0.0,0,0.0,0.00,0.0


### Link Labels to Stats

In [4]:
import numpy as np
def make_x(commit_stats):
    return [
        np.log1p(commit_stats['binFiles']),
        np.log1p(commit_stats['textFiles']),
        np.log1p(commit_stats['textLines']),
        np.log1p(commit_stats['javascriptLines']),
        np.log1p(commit_stats['rustLines']),
        np.log1p(commit_stats['lockLines']),
        np.log1p(commit_stats['htmlLines']),
        np.log1p(commit_stats['jsonLines']),
        np.log1p(commit_stats['markdownLines']),
        np.log1p(commit_stats['imgFiles']),
        np.log1p(commit_stats['yarnLines']),
        #np.log1p(commit_stats['clojureLines']),
        #np.log1p(commit_stats['shellLines']),
    ]


In [5]:
stats = {}
for commit in all_stats:
    stats[commit['commit']] = commit

X = []
Y = []
a = 0
for row in data.to_numpy():
    commit_id = row[0]
    row_labels = row[slice(2, 9)]
    x = make_x(stats[commit_id])
    #print(x)
    #print(a)
    X.append(make_x(stats[commit_id]))
    Y.append(row_labels.tolist())
    a += 1
#print(a)

#print(Y[slice(2)])
#print(Y[0][0])
#print(X[slice(2)])

In [6]:
from sklearn.datasets import make_multilabel_classification

sample_X, sample_y = make_multilabel_classification(n_classes=3, random_state=0)
print(sample_y[slice(2)])
print(Y[slice(2)])

[[0 1 0]
 [0 1 0]]
[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

print(len(X))
print(len(Y))

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X, Y)
score = clf.score(X, Y)
Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

160
160
0.84375


/home/bob/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.33, random_state=1)

clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(20, 20),
                    random_state=3, max_iter=10000)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
Y_pred = clf.predict(X)
print(score)
print(Y_pred)

0.7924528301886793
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [9]:
#print(X_test)
#print(Y_pred)
pairs = []
for i in range(0,len(X_test)):
    #print(Y_pred[i].tolist())
    pairs.append(Y_pred[i].tolist() + X_test[i])
# print(pairs)

print(len(X_train))
import pandas as pd
pd.DataFrame(pairs).to_csv("/home/bob/test.tsv", sep="\t")

107
